# 📘 Example: Database Builder

In this notebook, we demonstrate how to use the **Database builder** of FloodAdapt, which greatly simplifies the process of setting up a FloodAdapt database in a new location!

The most critical components of a FloodAdapt database are the [**SFINCS**](https://github.com/Deltares/SFINCS) and [**Delft-FIAT**](https://github.com/Deltares/Delft-FIAT) models, both of which can be generated with great ease using the [**HydroMT-SFINCS**](https://deltares.github.io/hydromt_sfincs/latest/) and the [**HydroMT-FIAT**](https://deltares.github.io/hydromt_fiat/stable/) plugins of [**HydroMT**](https://deltares.github.io/hydromt/latest/).

For this notebook, we will use an example area in Charleston, USA, for which we have already generated a SFINCS and a Delft-FIAT model.

In order to use the **DatabaseBuilder** of FloodAdapt a set of **configuration** parameters are needed. The **configuration** parameters can be divided to **mandatory** and **optional** ones. Using only the mandatory parameters (i.e., baseline FloodAdapt configuration) will result in a simple but functional version of FloodAdapt. By adding optional parameters to your configuration, you can create a more advanced FloodAdapt database with additional features. 

If you want to learn more about the configuration parameters, please refer to the [Database-Builder](../../3_setup_guide/database.qmd) of the Setup Guide in the documentation.

The configuration can be either created through available FloodAdapt classes or can be parsed as a simple dictionary. We advice you to work with the FloodAdapt classes, since this can avoid using wrong parameter names or values with the help of type hinting. 

In this notebook we will start by creating a simple FloodAdapt database using the baseline FloodAdapt configuration ([Step-2: Build a basic FloodAdapt database](#-step-2-build-a-basic-floodadapt-database)), and then we will go through all the optional configuration parameters and create a more complex database ([Step-4: Build an advanced FloodAdapt Database](#️-step-4-build-an-advanced-floodadapt-model)).

## Import libraries

In [ ]:
# Import packages
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import toml
from pathlib import Path
from shapely.geometry import Polygon
from hydromt_fiat.fiat import FiatModel
from hydromt_sfincs.sfincs import SfincsModel
import flood_adapt.adapter.fiat_adapter as fiat
import flood_adapt.database_builder.database_builder as db
from flood_adapt import FloodAdapt
from flood_adapt.objects.forcing.tide_gauge import TideGaugeSource
from flood_adapt.objects.forcing.timeseries import Scstype
from flood_adapt.config.config import Settings
from flood_adapt import unit_system as us
%matplotlib inline

## 🔍 **Step 1**. Explore the SFINCS and Delft-FIAT model

The study area is in **Charleston, USA**, a coastal city on the East Coast of the United States. 

We can first inspect the extents of our SFINCS model, by loading the model with the HydroMT-SFINCS plugin.

In [ ]:
# Define the static data folder
STATIC_DATA_DIR = Path("../../../_data/examples/static-data/1_DatabaseBuilder").resolve()

fn_sfincs = STATIC_DATA_DIR  / "overland"

sfincs = SfincsModel(root=str(fn_sfincs), mode="r")
sfincs.read()
gdf = sfincs.region[["geometry"]]
gdf["name"] = "SFINCS Model Extent"
gdf.explore(
    style_kwds={"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2},
    tiles="CartoDB positron",
    column="name",
    legend=True,
    legend_kwds={"caption": "Region"}
)

Then we can inspect the exposure objects (buildings and roads) of the Delft-FIAT model, by loading the model with the HydroMT-FIAT plugin.

In [ ]:
fn_fiat = STATIC_DATA_DIR  / "fiat"

fiat = FiatModel(root=str(fn_fiat), mode="r")
fiat.read()

# Get the geodataframe with exposure data
gdf = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)

# Plot the region and the secondary_object_types of the exposure data
gdf.explore(column="secondary_object_type", 
                name="Exposure types",
                tiles="CartoDB positron"
                )

## 📄 **Step 2**: Build a **basic** FloodAdapt Database

In this step we will create a basic FloodAdapt database, using only the mandatory configuration parameters.  

### 📚 **Step 2a**: Build the configuration from the **Database Builder classes**

To create the configuration object we can use the `ConfigModel` class of the Database Builder.

The mandatory configuration attributes include the `name` of the database and the `database_path` which points to the location where the database will be stored. Then, a `unit_system` needs to be specified, which can be either `metric` or `imperial`. The `unit_system` will determine the default units used in the database. For the output visualizations, scaling values need to be specified for each output type, using the `gui` attribute. 

Last, the overland SFINCS model and the Delft-FIAT model need to be specified. The SFINCS model is specified using the `sfincs_overland` attribute, which includes the path to the SFINCS model and the vertical reference that the model has. The Delft-FIAT model is specified using the `fiat` attribute, which points to the path of the Delft-FIAT model.

In [ ]:
# TODO change gui attribute to visualization
# TODO change FloodModel name to path

# Initiate the config_model for a simple FloodAdapt Database configuration file 
config_model = db.ConfigModel(
    name="charleston_example_basic",
    database_path= str((STATIC_DATA_DIR /"Database").absolute().as_posix()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    sfincs_overland=db.FloodModel(
        name=str((STATIC_DATA_DIR  / "overland").absolute().as_posix()),
        reference="MSL",
    ),
    fiat=str((STATIC_DATA_DIR / "fiat").absolute().as_posix()),
)

### 📖 **Step 2b**: Create a configuration file from a **dictionary**

An alternative approach, would be to create a dictionary with all the attributes and save it to a TOML file. Any path that is included in the file should be either an absolute path (using forward slashes `/`) or a relative path (using forward slashes `/` and relative to the path of the TOML file).

In [ ]:
# In this case we use relative paths, but absolute paths are also possible
config_dict = {
"name": "charleston_example_basic",
"database_path": "Database",
"sfincs_overland" : { 
    "name":"overland",
    "reference":"MSL"
    },
"fiat" : "fiat",
"unit_system" :"imperial",
"gui": {
"max_flood_depth": 5,
"max_aggr_dmg" : 1e6,
"max_footprint_dmg": 250000,
"max_benefits" : 5e6}
}

We can then save the configuration as a .toml file at the location of the other input data, that we defined in the dictionary as relative paths.

In [ ]:
# Save the configuration file
config_path = STATIC_DATA_DIR  / "db_config_basic.toml"
with open(config_path, "w") as f:
    toml.dump(config_dict, f)

The we can use the `read()` method of the `ConfigModel` class to read the configuration from the TOML file.

In [ ]:
config = db.ConfigModel.read(config_path)

We can then verify that the

In [ ]:
config == config_model

## 🏃‍♀️ **Step 3**: Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 2a** - in which we generated the configuration using the FloodAdapt classes.

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

Now you created two complete FloodAdapt Database. Both databases should be identical as we used the same inputs. You can open the databases in the GUI and explore them further or continue working with the database through the API. 

## 🗂️ **Step 4**. Build an **advanced** FloodAdapt model

In the previous step we created a simple FloodAdapt model. Our simple database is limited in functionality, so in this next step we're adding more advanced configurations to expand its capabilities so that we can create a comprehensive FloodAdapt model. 

The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

### 📚 **Step 2a**: Build the configuration from the **Database Builder classes**

First, we need to define the basic parameters like the `name` and `database_path`, together with the `unit_system` as an `UnitSystem` object and the `gui` variables, which we aggregate in the `GuiConfigModel`.

In [ ]:
# Set the basic parameters
name="charleston_example_advanced1"
database_path= str((STATIC_DATA_DIR / "Database").absolute())
unit_system= db.UnitSystems.imperial
gui=db.GuiConfigModel(
    max_flood_depth=5,
    max_aggr_dmg=1e6,
    max_footprint_dmg=250000,
    max_benefits=5e6,
)

Additionally, to the basic model parameters we can add more parameters to make the database more complex. 

**Risk**   
We can add a probabilistic event set by providing the filepath the risk event in the attribute `probabilistic_set`. If we add risk to our database we can set the `infographics` to `True`.
We also need to define the `return_periods` in form of a list of integers or floats.

In [ ]:
# Add risk parameters
probabilistic_set=str(Path(STATIC_DATA_DIR  / "test_set"))
infographics=True
return_periods=[1, 2, 5, 10, 25, 50, 100]

We need to define the **sfincs model(s)** for our database. We need to add the `sfincs_overland` model by passing a dictionary with the entries: `name`, the file path,  and `reference`, the reference system (eg. "MSL").
 
If we have a SFINCS offshore model we can also pass this into the configuration to the `sfincs_offshore` attribute in he same way as the overland model. For the offshore model we need to add an extra entry to the dictionary. We need to add the value and unit of the `vertical_offset`, which describes the offshore water level. 

In [ ]:
# Add the sfincs model(s)
sfincs_overland=db.FloodModel(
    name=str((STATIC_DATA_DIR  / "overland").absolute()),
    reference="MSL",
)
sfincs_offshore=db.FloodModel(
    name=str(Path(STATIC_DATA_DIR / "offshore")),  #TODO fix final path
    reference="MSL",
    vertical_offset=us.UnitfulLength(
        value=0.33, units=us.UnitTypesLength.feet
    ))

Next we must pass a **DEM** in form of a dictionary with the entries: `filename` and `units`. The `units` should be passed as [`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd) object. 

In [ ]:
# Add the DEM
dem=db.DemModel(
    filename=str(Path(STATIC_DATA_DIR / "charleston_14m.tif")), 
    units=us.UnitTypesLength.meters,
)

We can also exclude specific datums which we can define in the `excluded_datums` attribute. We can pass a list of datums. <span style="color:red;">Why exclude datums? What does that mean in this context</span>

**Important!**  <span style="color:red;">double check</span>
The water level reference should be set to the reference of your DEM. You can create a reference system manually as shown below or fetch this information from a close-by tide gauge. This will be shown in the more advanced options. 

In [ ]:
# Add the reference and exclude datums
excluded_datums=["NAVD88"]
#references=db.WaterlevelReferenceModel(
#    reference="MSL",
#    datums=[
#        db.DatumModel(name="MSL", height=us.UnitfulLength(value=0, units=us.UnitTypesLength.meters)),
#        db.DatumModel(name="NAVD88", height=us.UnitfulLength(value=1, units=us.UnitTypesLength.meters))
#    ]
#)

We can add **sea level rise scenarios** from a csv file wit the columns:  **year, unit, scenario_1, scenario_2, ..., scenario_n**. With that file we create a `SlrScenariosModel` object in which we provide the `file` and the `relative_to_year` attributes.

In [ ]:
# Add SLR scenarios
slr_scenarios=db.SlrScenariosModel(
    file=str(Path(STATIC_DATA_DIR  / "slr.csv")),
    relative_to_year=2020,
)

Let's have a quick look what the slr scenario csv file looks like.

In [ ]:
pd.read_csv(slr_scenarios.file)

In a similar manner as the slr scenarios, we can add **SCS (Soil Conservation Service) rainfall** to our database by creating a `SCSModel` object, which consist of the `file` and the `type` (Scs type) attributes. The `type` should be passed in form of a `Scstype` object.

In [ ]:
# Add Soil Conservation Service rainfall
scs=db.SCSModel(
    file=str(Path(STATIC_DATA_DIR / "scs_rainfall.csv")),
    type=Scstype.type3,
)

Let's have a quick look what the scs scenario csv file looks like.

In [ ]:
pd.read_csv(scs.file)

To capture a realistic tide we can add the information from a **tide gauge**. We can either download the data from the NOAA COOPS or pass a csv file ino the `source` attribute of the `TideGaugeConfigModel`. When we download data from NOAA COOPS, we must define a `max_distance` of object type [`UnitfulLength`](../../api_ref/UnitfulLength.qmd) that describes the value (`int`) and unit ([`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd)) of the maximum distance from our model domain to the nearest tide gauges to include.  

In [ ]:
# Add tide gauges
tide_gauge=db.TideGaugeConfigModel(
    source=db.TideGaugeSource.noaa_coops,
    max_distance=us.UnitfulLength(
        value=100, units=us.UnitTypesLength.miles
    ),
)

By setting the `cyclones` attribute to `True` we can add **cyclone tracks** to our database. We need to define the ocean basin we are interested in in the `cyclone_basin` attribute. The `Basins` object already has several options for us e.g. `NA` - North Atlantic. 

In [ ]:
# Add cyclone tracks
cyclones=True
cyclone_basin=db.Basins.NA

By adding **observation points** we can extract timeseries of water levels from our event scenarios. We can add a list of `ObsPointModel` objects. Each of these objects must have a `name` and a `lat`and `lon`attribute. The `description` and `ID` are optional.

In [ ]:
# Add observation points
obs_point=[
    db.ObsPointModel(
        name="ashley_river",
        description="Ashley River - James Island Expy",
        lat=32.7765,
        lon=-79.9543,
    ),
    db.ObsPointModel(
        name=8665530,
        description="Charleston Cooper River Entrance",
        ID=8665530,
        lat=32.78,
        lon=-79.9233,
    ),
]

We need to define the **Delft-FIAT model** for our database. All we need for that is to define the folder path that points to the Delft-FIAT model.

In [ ]:
# Add the Delft-FIAT model
fiat=str(Path(STATIC_DATA_DIR / "fiat").absolute())

 <span style="color:red;">Here I have a question - this is only used when there are NO aggregation areas in our model yet? Bc when I run it with an existing aggr layer in the model this crashes.</span>
- `aggregation_areas`: A list of dictionaries with the entries: name, file path, field_name. Aggregates the exposure into larger spatial groups o summarize impacts on larger scale. 

In [ ]:
# Add aggregation areas
#aggregation_areas=[
#db.SpatialJoinModel(
#    name="aggr_lvl_1",
#    file=str(
#        Path(STATIC_DATA_DIR / aggr_lvl_1.geojson")
#    ),
#    field_name="name",
#),
#db.SpatialJoinModel(
#    name="aggr_lvl_2",
#    file=str(Path(STATIC_DATA_DIR /aggr_lvl_2.geojson")
#    ),
#    field_name="name",
#),
#]

To refine our Delft-FIAT model we can add additional inputs or update default settings. 

Sometimes our Delf-FIAT model exposure is in point data By adding the `building_footprints`attribute we can download data from Open Street Map (OSM) using the `FootprintsOptions.OSM` object. e use the building footprints to create the visualizations for the impacts on the building footprint level.  
The `fiat_buildings_name` and `fiat_roads_name` are set to *"buildings"* and *"roads"*, respectively as default. These names capture the geometry names of these assets in the Delft-FIAT model. If you specified different names for these geometry, you must change them here. By defining a value in the `road_width` attribute we can capture the realistic width of the road. 

In [ ]:
# Add additional Delft-FIAT parameters
building_footprints=db.FootprintsOptions.OSM
fiat_buildings_name="buildings"
fiat_roads_name="roads"
road_width=5

The base flood elevation (BFE) model allows us to elevate homes relative to the BFE. To do so we need to provide a BFE spatial file to the `SpatialJoinModel`, defining the `name` of the spatial file, in this case `bfe` and the `field_name`, which is the column name inside the spatial file that defines the BFE value.

In [ ]:
# Add base flood elevation
bfe=db.SpatialJoinModel(
    file=str(Path(STATIC_DATA_DIR / "bfe.geojson")),
    name="bfe",
    field_name="bfe",
)

To capture socio-economic impacts we can add a social vulnerability (SVI) layer to the database. We need to define the `file`-path to the spatal SVI file, the `field_name` that captures the column name within the spatial file with the SVI value and the `threshold` at which point an area is classified as vulnerable.   

In [ ]:
# Add social vulnerability index
svi=db.SviConfigModel(
    file=str(Path(STATIC_DATA_DIR / "CDC_svi_2020.gpkg")),
    field_name="SVI",
    threshold=0.5,
)

Now, that we created all the individual objects we can compile them in the `ConfigModel`. 

In [ ]:
# Compile ConfigModel
config_model = db.ConfigModel(name = name,
        database_path= database_path,
        unit_system= unit_system,
        gui=gui,
        infographics=infographics,
        probabilistic_set=probabilistic_set,
        return_periods=return_periods,
        sfincs_overland=sfincs_overland,
        sfincs_offshore=sfincs_offshore,
        dem=dem,
        excluded_datums=excluded_datums,
        #references=references,
        slr_scenarios=slr_scenarios,
        scs=scs,
        tide_gauge=tide_gauge,
        cyclones=cyclones,
        cyclone_basin=cyclone_basin,
        obs_point=obs_point,
        fiat=fiat,
        #aggregation_areas=aggregation_areas,
        building_footprints=db.FootprintsOptions.OSM,
        fiat_buildings_name=fiat_buildings_name,
        fiat_roads_name=fiat_roads_name,
        bfe=bfe,
        svi=svi,
        road_width=road_width,
)

### 📖 **Step 4b**: Create a configuration file from a **dictionary**
From verything that we learned above, we can create a simple dictionary with all the attributes and save it to a configuration file.

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 

# all paths should be provided with forward slashes (/)
config = {"name": "charleston_example_advanced2",
 "database_path": "Database",
 "unit_system": "imperial",
 "gui": {"max_flood_depth": 5.0,
  "max_aggr_dmg": 1000000.0,
  "max_footprint_dmg": 250000.0,
  "max_benefits": 5000000.0},
 "infographics": True,
 "fiat": "fiat",
 #"aggregation_areas": [{"name": "aggr_lvl_1",
 #  "file": "aggr_lvl_1.geojson",
 #  "field_name": "name"},
 # {"name": "aggr_lvl_2",
 #  "file": "aggr_lvl_2.geojson",
 #  "field_name": "name"}],
 "building_footprints": "OSM",
 "fiat_buildings_name": "buildings",
 "fiat_roads_name": "roads",
 "bfe": {"name": "bfe",
  "file": str(Path(STATIC_DATA_DIR /"bfe.geojson")),
  "field_name": "bfe"},
 "svi": {"name": None,
  "file": str(Path(STATIC_DATA_DIR /"CDC_svi_2020.gpkg")),
  "field_name": "SVI",
  "threshold": 0.5},
 "road_width": 5.0,
 "return_periods": [1, 2, 5, 10, 25, 50, 100],
 "sfincs_overland": {"name": "overland",
  "reference": "MSL",
  "vertical_offset": None},
 "sfincs_offshore": {"name": "offshore",
  "reference": "MSL",
  "vertical_offset": {"value": 0.33, "units":"feet"}},
 "dem": {"filename": str(Path(STATIC_DATA_DIR / "charleston_14m.tif")),
  "units": "meters"},
 "excluded_datums": ["NAVD88"],
 "slr_scenarios": {"file": "slr.csv",
  "relative_to_year": 2020},
 "scs": {"file": str(Path(STATIC_DATA_DIR / "scs_rainfall.csv")),
  "type": "type_3"},
 "tide_gauge": {"source":"noaa_coops",
  "description": "",
  "max_distance": {"value": 100.0, "units": "miles"}},
 "cyclones": True,
 "cyclone_basin": "NA",
 "obs_point": [{"name": "ashley_river",
   "description": "Ashley River - James Island Expy",
   "lat": 32.7765,
   "lon": -79.9543},
  {"name": 8665530,
   "description": "Charleston Cooper River Entrance",
   "ID": 8665530,
   "lat": 32.78,
   "lon": -79.9233}],
 "probabilistic_set": "test_set"}

When we work with a dictionary, we need to save the configuration file as a .toml-file, so that we can call it later when we run the Database Builder.

In [ ]:
# Save the configuration file
with open(Path(STATIC_DATA_DIR /"db_config_advanced.toml"), "w") as f:
    toml.dump(config, f)

## 🏃‍♀️ **Step 5**. Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 3a** - in which we generated the configuration using the FloodAdapt classes.

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

Now we are going to run **Option 3b** - in which we manually created a configuration file from a dictionary.

In [ ]:
# Run Option 2b -  DB-builder config from dictionary
config_path = STATIC_DATA_DIR  / "db_config_advanced.toml"
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

## 🚀 **Step 6.** - Reading-in the FloodAdapt database
Now that we built the database we can open it and continue to work with it.  

In the other example notebooks in this repository you can find the instructions on how to create and save the single components to create a full scenario (events, measures, strategies, projections) in your database.

In [ ]:
settings = Settings(
    DATABASE_ROOT=Path(STATIC_DATA_DIR / "Database").resolve(),
    DATABASE_NAME="charleston_example_advanced1"
)
fa = FloodAdapt(database_path=settings.database_path)

## **Finished!**

Congratulations! You created your own FloodAdapt database and know now how to initiate it. 